<a href="https://colab.research.google.com/github/comparativechrono/Rephasing-of-Seasonal-Birth-Rates-in-the-United-Kingdom-/blob/main/Supplemental_material/SI_table_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime
from statsmodels.tsa.seasonal import STL

# ---------------------------------------------------------------------------
# 1. Load data and parse two-digit years
# ---------------------------------------------------------------------------

CSV_FILE = "environmental.csv"       # <-- put file in same folder

def parse_date_two_digit(s: str) -> pd.Timestamp:
    """Convert 'Mon-yy' to Timestamp; 00-25 ⇒ 2000-2025, else 1900-1999."""
    month_str, year_str = s.split('-')
    month = datetime.strptime(month_str, '%b').month
    yy    = int(year_str)
    year  = yy + 2000 if yy <= 25 else yy + 1900
    return pd.Timestamp(year=year, month=month, day=1)

df = pd.read_csv(CSV_FILE)
df["Date"] = df["Date"].apply(parse_date_two_digit)
df.set_index("Date", inplace=True)

# ---------------------------------------------------------------------------
# 2. Parameters
# ---------------------------------------------------------------------------

env_vars   = ["light", "photoperiod", "temp"]
window     = 60           # rolling window length (months)
max_lag    = 20           # look up to 20 months back
lags       = np.arange(-max_lag, 0)   # negative only (env leads BR)
dates      = df.index[window-1:]      # window-end dates

# ---------------------------------------------------------------------------
# 3. Helper: peak-lag time-series with unequal-n weighting
# ---------------------------------------------------------------------------

def peak_lag_series(x: np.ndarray, y: np.ndarray,
                    lags: np.ndarray, win: int) -> np.ndarray:
    """
    For each window return the lag giving the largest |t-stat|, where
        t = r * sqrt(n_eff-2) / sqrt(1-r²)
    We optimise the monotonic part |r|*sqrt(n_eff-2), which is sufficient
    for ranking lags and cheaper to compute.
    """
    n_windows = len(x) - win + 1
    peak_lags = np.empty(n_windows, dtype=int)

    for i in range(n_windows):
        win_x, win_y = x[i:i+win], y[i:i+win]
        best_score, best_lag = -np.inf, lags[0]

        for lag in lags:      # all negative
            shift = -lag
            xi, yi = win_x[:-shift], win_y[shift:]
            n_eff  = len(xi)
            if n_eff < 3:
                continue
            r = np.corrcoef(xi, yi)[0, 1]
            if np.isnan(r):
                continue
            score = abs(r) * np.sqrt(n_eff - 2)
            if score > best_score:
                best_score, best_lag = score, lag

        peak_lags[i] = best_lag

    return peak_lags

# ---------------------------------------------------------------------------
# 4. Peak lags on raw data
# ---------------------------------------------------------------------------

raw_peak_lags = {
    var: peak_lag_series(df[var].values, df["BR"].values, lags, window)
    for var in env_vars
}

# ---------------------------------------------------------------------------
# 5. Deseasonalise via STL and repeat
# ---------------------------------------------------------------------------

df_ds = pd.DataFrame(index=df.index)
for col in env_vars + ["BR"]:
    stl = STL(df[col], period=12, robust=True).fit()
    df_ds[col] = df[col] - stl.seasonal

ds_peak_lags = {
    var: peak_lag_series(df_ds[var].values, df_ds["BR"].values, lags, window)
    for var in env_vars
}

# ---------------------------------------------------------------------------
# 6. Define study periods
# ---------------------------------------------------------------------------

mask55_74 = (dates.year >= 1955) & (dates.year <= 1974)
mask76_14 = (dates.year >= 1976) & (dates.year <= 2014)

# ---------------------------------------------------------------------------
# 7. Regression helper
# ---------------------------------------------------------------------------

def regress_period(peak_dict, mask, period_label):
    t_rel = (dates[mask] - dates[mask][0]).days / 365.25  # years since start
    rows  = []

    for v in env_vars:
        y = peak_dict[v][mask]
        X = sm.add_constant(t_rel)
        model = sm.OLS(y, X).fit()
        rows.append({
            "Variable":          v,
            "Slope (months/yr)": model.params[1],
            "StdErr":            model.bse[1],
            "p-value":           model.pvalues[1],
            "N windows":         len(y)
        })

    tbl = pd.DataFrame(rows)
    print(f"\n=== {period_label} ===")
    print(tbl.to_string(index=False))
    return tbl

# ---------------------------------------------------------------------------
# 8. Run regressions and print tables
# ---------------------------------------------------------------------------

regress_period(raw_peak_lags, mask55_74, "1955-1974  (raw)")
regress_period(ds_peak_lags,  mask55_74, "1955-1974  (deseasonalised)")
regress_period(raw_peak_lags, mask76_14, "1976-2014  (raw)")
regress_period(ds_peak_lags,  mask76_14, "1976-2014  (deseasonalised)")
